# BFD selection

In [2]:
import astropy.io.fits as fits
from bfd import TierCollection
import numpy as np
import healsparse

# load targets
targets = fits.open('/global/cfs/cdirs/des/mgatti/BFD_targets_newrun/targets.fits')
targets_extrainfo = fits.open('/global/cfs/cdirs/des/mgatti/BFD_targets_newrun/targets_extrainfo.fits')

# load noisetiers
tc = TierCollection.load('/global/cfs/cdirs/des/mgatti/BFD_targets_newrun//noisetiers.fits')
noisetier = fits.open('/global/cfs/cdirs/des/mgatti/BFD_targets_newrun/noisetiers.fits')

# 
noisetier_targets = targets[1].data['NOISETIER']
Mf = targets[1].data['moments'][:,0]
covariance = targets[1].data['covariance']
ra = targets_extrainfo[1].data['ra']
dec = targets_extrainfo[1].data['dec']

unique_noisetier = np.unique(noisetier_targets)

# selection 1 ----
# (exclude large objects)
mask = (targets[1].data['moments'][:,1]/targets[1].data['moments'][:,0] > 2.2) 

# Assign noisetiers. In the targets catalog, this is already done ("noistetier_targets")
# but this code shows you how to do it for other objects.
nt_2 = tc.assign(covariance)
noisetier_targets_2 = -1*np.ones(covariance.shape[0]).astype(np.int)
for key in nt_2.keys():
    noisetier_targets_2[nt_2[key]]=key
    
# noistetier_targets_2 will be the same as noistetier_targets.


# SN selection ---
# The SN selection is applied tier-by-tier. We  use the targets' flux moment and a tier mean flux covariance.

sn_min = 7
sn_max = 200

mask_sn  = np.array([False]*len(noisetier_targets))
for tier in range(20):
    mask_nt = noisetier_targets == tier

    mean_cov_mf = (noisetier[1+tier].data['COVARIANCE'][0,0])
    min_flux = sn_min * np.sqrt(mean_cov_mf)
    max_flux = sn_max * np.sqrt(mean_cov_mf)
    mask_sn[mask_nt] = (Mf[mask_nt] > min_flux) & (Mf[mask_nt] < max_flux)
    
    
# hleda mask
fname = '/global/cfs/cdirs/des/y6-shear-catalogs/y6-combined-hleda-gaiafull-des-stars-hsmap16384-nomdet-v3.fits'
hmap = healsparse.HealSparseMap.read(fname)
in_footprint = hmap.get_values_pos(
        ra,dec, valid_mask=True)

# mask tot
mask_tot = mask_sn & mask & in_footprint  
len(mask_tot[mask_tot]) 

[12  2] [0 0 0 ... 0 0 0] [8 6 8 ... 2 2 2]
Tier 0 gets 76722 targets
Tier 1 gets 3722 targets
Tier 2 gets 697541 targets
Tier 3 gets 52949 targets
Tier 4 gets 6279464 targets
Tier 5 gets 631812 targets
Tier 6 gets 9505813 targets
Tier 7 gets 1895860 targets
Tier 8 gets 3457998 targets
Tier 9 gets 1273971 targets
Tier 10 gets 760470 targets
Tier 11 gets 387662 targets
Tier 12 gets 239435 targets
Tier 13 gets 126922 targets
Tier 14 gets 109949 targets
Tier 15 gets 55156 targets
Tier 16 gets 46721 targets
Tier 17 gets 27733 targets
Tier 18 gets 17597 targets
Tier 19 gets 14960 targets
Tier 20 gets 3966 targets
Tier 21 gets 4061 targets
Tier 22 gets 1632 targets
Tier 23 gets 3084 targets


/tmp/ipykernel_202192/3211127281.py:29: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  noisetier_targets_2 = -1*np.ones(covariance.shape[0]).astype(np.int)
